# Preprocessing NaN values
### for the 7 last columns
    17  Можно с детьми/животными  17272 non-null  object
        Create columns: "pets_okay" (and fill NA), "kids_okay" (and fill NA)
    18  Дополнительно             23011 non-null  object
        Create columns for potentially impactful factors: Мебель в комнатах (and count, then fill NA if needed or drop the column), Кондиционер (likewise), Посудомоечная машина (likewise)
    15  Окна                      16755 non-null  object
        Values: categorical, [На улицу и двор' 'Во двор' 'На улицу']. Create "only_street_view"
    16  Санузел                   20696 non-null  object
        Values: need to make them numerical and categorical ("total_bathrooms", "only_one"), then fill NaN.
    22  Лифт                      17868 non-null  object
        Create columns: "has_elevator", "has_cargo_elevator"
    21  Высота потолков, м        11206 non-null  float64
        Remove outliers, transform into meters, fill NaN, consider creating categorical (low, medium, high)
    23  Мусоропровод              12846 non-null  object
        Fill NA.

In [1]:
import pandas as pd

import numpy as np
from numpy import mean
from numpy import std
from numpy import ravel

import seaborn as sns
import matplotlib.pyplot as plt

import scipy
import joblib
import re
import glob
import sklearn
from math import sqrt
from decimal import Decimal

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
# from sklearn.metrics import mean_absolute_percentage_error


from geopy.exc import GeocoderTimedOut
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

from tqdm import tqdm

# !pip install CurrencyConverter
from currency_converter import CurrencyConverter

In [2]:
df = pd.read_csv('../data/_data.csv')

## Column "Можно с детьми/животными"

####  New column "Можно с животными"

In [3]:
df['pets_okay'] = df['Можно с детьми/животными'].apply(lambda x: 1 if isinstance(x, str) and 'Можно с животными' in x else (0 if isinstance(x, str) else np.nan))
mode_pets_okay = df['pets_okay'].mode()[0]
df['pets_okay'].fillna(mode_pets_okay, inplace=True)
print(df['pets_okay'])

0        1.0
1        0.0
2        0.0
3        1.0
4        0.0
        ... 
23363    0.0
23364    0.0
23365    0.0
23366    0.0
23367    1.0
Name: pets_okay, Length: 23368, dtype: float64


In [4]:
percent_counts = df['pets_okay'].value_counts(normalize=True) * 100

# Print the percentage counts
print("Percentage of each value in 'pets_okay':")
print(percent_counts)

Percentage of each value in 'pets_okay':
pets_okay
0.0    69.453954
1.0    30.546046
Name: proportion, dtype: float64


####  New column "Можно с детьми"

In [5]:
df['kids_okay'] = df['Можно с детьми/животными'].apply(lambda x: 1 if isinstance(x, str) and 'Можно с детьми' in x else (0 if isinstance(x, str) else np.nan))
mode_kids_okay = df['kids_okay'].mode()[0] # 🚧
df['kids_okay'].fillna(mode_kids_okay, inplace=True)
print(df['kids_okay'])

0        1.0
1        1.0
2        1.0
3        0.0
4        1.0
        ... 
23363    1.0
23364    1.0
23365    1.0
23366    1.0
23367    1.0
Name: kids_okay, Length: 23368, dtype: float64


In [6]:
percent_counts = df['kids_okay'].value_counts(normalize=True) * 100

# Print the percentage counts
print("Percentage of each value in 'kids_okay':")
print(percent_counts)

Percentage of each value in 'kids_okay':
kids_okay
1.0    98.977234
0.0     1.022766
Name: proportion, dtype: float64


## Column "Дополнительно"
#### Create columns for potentially impactful factors: Мебель в комнатах (and count, then fill NA if needed or drop the column), Кондиционер (likewise), Посудомоечная машина (likewise)

In [7]:
def create_column(df, new_col_name, phrase):
    df[new_col_name] = df['Дополнительно'].apply(
        lambda x: 1 if isinstance(x, str) and phrase in x else (0 if isinstance(x, str) else np.nan)
    )
    # Fill NaN values with the mode
    mode_value = df[new_col_name].mode()[0]
    df[new_col_name].fillna(mode_value, inplace=True)
    return df

# Create new columns
df = create_column(df, 'furniture_in_the_room', 'Мебель в комнатах')
df = create_column(df, 'air_conditioner', 'Кондиционер')
df = create_column(df, 'dishwashing', 'Посудомоечная машина')
df = create_column(df, 'fridge', 'Холодильник')
df = create_column(df, 'internet', 'Интернет')

In [8]:
# Checking descriptive stats for each main feature
def calculate_percentage(df, column_name):
    percent_counts = df[column_name].value_counts(normalize=True) * 100
    print(f"Percentage of each value in '{column_name}':")
    print(percent_counts)

calculate_percentage(df, 'furniture_in_the_room')
calculate_percentage(df, 'air_conditioner')
calculate_percentage(df, 'dishwashing')
# Doing reality check - these columns should largely have 1 
calculate_percentage(df, 'fridge')
calculate_percentage(df, 'internet')


print(df[['furniture_in_the_room', 'air_conditioner', 'dishwashing', 'fridge', 'internet']])


Percentage of each value in 'furniture_in_the_room':
furniture_in_the_room
1.0    92.643786
0.0     7.356214
Name: proportion, dtype: float64
Percentage of each value in 'air_conditioner':
air_conditioner
0.0    62.358781
1.0    37.641219
Name: proportion, dtype: float64
Percentage of each value in 'dishwashing':
dishwashing
0.0    66.48836
1.0    33.51164
Name: proportion, dtype: float64
Percentage of each value in 'fridge':
fridge
1.0    88.124786
0.0    11.875214
Name: proportion, dtype: float64
Percentage of each value in 'internet':
internet
1.0    80.498973
0.0    19.501027
Name: proportion, dtype: float64
       furniture_in_the_room  air_conditioner  dishwashing  fridge  internet
0                        1.0              1.0          1.0     1.0       1.0
1                        1.0              1.0          1.0     1.0       1.0
2                        1.0              1.0          1.0     1.0       1.0
3                        1.0              1.0          1.0     1.0      

## Column "Окна"
#### Create column only_street_view with supposedly negative impact


In [9]:
df['only_street_view'] = df['Окна'].apply(
    lambda x: 1 if isinstance(x, str) and 'На улицу' in x else (0 if isinstance(x, str) else np.nan)
)
mode_only_street_view = df['only_street_view'].mode()[0]
df['only_street_view'].fillna(mode_only_street_view, inplace=True)
percent_counts = df['only_street_view'].value_counts(normalize=True) * 100

print("Percentage of each value in 'only_street_view':")
print(percent_counts)



Percentage of each value in 'only_street_view':
only_street_view
0.0    74.815988
1.0    25.184012
Name: proportion, dtype: float64


## Column "Высота потолков"
#### Values: transform into meters, ❗️ Remove outliers, fill NaN, consider creating ❗️ categorical levels (low, medium, high)


In [10]:
# Function to convert meters to centimeters
def convert_to_cm(value):
    try:
        value = float(value)
        if value < 10:
            return value * 100
        return value
    except ValueError:
        return np.nan


print(convert_to_cm(2.63))



263.0


In [11]:
# Apply the function to convert all values to centimeters
df['ceiling'] = df['Высота потолков, м'].apply(convert_to_cm)

print(min(df['ceiling']))

12.8


In [12]:
# Проверка
def show_unique_values(data, column_name):
    unique_values = data[column_name].dropna().unique()  # Remove NaN values before sorting
    sorted_values = sorted(unique_values, reverse=True)
    print(f"Unique values for '{column_name}' in descending order:", sorted_values)

show_unique_values(df, 'ceiling')

Unique values for 'ceiling' in descending order: [900.0, 600.0, 580.0, 550.0, 509.99999999999994, 500.0, 470.0, 459.99999999999994, 450.0, 420.0, 415.00000000000006, 409.99999999999994, 405.0, 400.0, 392.0, 390.0, 380.0, 370.0, 360.0, 356.0, 355.0, 350.0, 341.0, 340.0, 335.0, 330.0, 325.0, 324.0, 321.0, 320.0, 315.0, 311.0, 310.0, 308.0, 307.0, 305.0, 302.0, 300.0, 299.0, 297.0, 295.0, 294.0, 293.0, 290.0, 288.0, 287.0, 286.0, 285.0, 284.0, 281.0, 280.0, 278.0, 277.0, 276.0, 275.0, 274.0, 273.0, 272.0, 270.0, 268.0, 267.0, 266.0, 265.0, 264.0, 263.0, 262.0, 260.0, 258.0, 257.0, 256.0, 254.99999999999997, 254.0, 252.0, 250.99999999999997, 250.0, 248.0, 245.00000000000003, 243.00000000000003, 240.0, 229.99999999999997, 220.00000000000003, 210.0, 200.0, 120.0, 28.0, 27.0, 26.0, 25.0, 12.8]


### Outliers

In [13]:
# Calculate IQR for "ceiling"
Q1 = df['ceiling'].quantile(0.25)
Q3 = df['ceiling'].quantile(0.95)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Lower bound for outliers: {lower_bound}")
print(f"Upper bound for outliers: {upper_bound}")

Lower bound for outliers: 180.0
Upper bound for outliers: 404.0


In [14]:
# Identify outliers
outliers = (df['ceiling'] < lower_bound) | (df['ceiling'] > upper_bound)
print(f"Number of outliers: {outliers.sum()}")

# Calculate and print the median value for verification
median_height = df.loc[~outliers, 'ceiling'].median() #🚧 tilda for substraction
print(f"Calculated median height: {median_height}")

# Replace outliers and NaNs with the median value
df.loc[outliers, 'ceiling'] = median_height #🚧 
df['ceiling'].fillna(median_height, inplace=True)

Number of outliers: 60
Calculated median height: 264.0


In [15]:

print(df['ceiling'].describe())

nan_count_after = df['ceiling'].isna().sum()
print(f"Number of NaN values in the 'ceiling' column after replacement: {nan_count_after}")

count    23368.000000
mean       269.332634
std         17.199493
min        200.000000
25%        264.000000
50%        264.000000
75%        264.000000
max        400.000000
Name: ceiling, dtype: float64
Number of NaN values in the 'ceiling' column after replacement: 0


### Transform into ordinal

In [16]:
# Define thresholds for categorizing ceiling heights (in centimeters) based on distribution and human comfort
low_ceiling_threshold = 259  # Uncomfortably low ceilings
high_ceiling_threshold = 300  # Spaciously high ceilings

# Print the percentage of values below the low threshold and above the high threshold
low_percentage = (df['ceiling'] < low_ceiling_threshold).mean() * 100
high_percentage = (df['ceiling'] > high_ceiling_threshold).mean() * 100
print(f"Percentage of uncomfortably low ceilings: {low_percentage}%")
print(f"Percentage of spaciously high ceilings: {high_percentage}%")

Percentage of uncomfortably low ceilings: 5.263608353303663%
Percentage of spaciously high ceilings: 4.732968161588498%


In [17]:
# Categorize ceiling heights
def categorize_ceiling(height):
    if height < low_ceiling_threshold:
        return 1  # Uncomfortably low
    elif height <= high_ceiling_threshold:
        return 2  # Medium
    else:
        return 3  # Spaciously high

# Apply the categorization
df['ceiling_ranking'] = df['ceiling'].apply(categorize_ceiling).astype(int)

# Display the distribution of the new categories
ceiling_category_distribution = df['ceiling_ranking'].value_counts()
print("Ceiling ranking distribution:")
print(ceiling_category_distribution)

Ceiling ranking distribution:
ceiling_ranking
2    21032
1     1230
3     1106
Name: count, dtype: int64


## Column "Санузел"
#### Values: need to make them numerical and categorical ("total number", "only совмещенный"), then fill NaN.


In [22]:
freq_table = df['Санузел'].value_counts(dropna=False).sort_values(ascending=False)

# Print the frequency table
print(freq_table)

Санузел
Совмещенный (1)                    10078
Раздельный (1)                      7158
NaN                                 2672
Совмещенный (2)                     1437
Совмещенный (1), Раздельный (1)      812
Раздельный (2)                       534
Совмещенный (3)                      241
Совмещенный (2), Раздельный (1)      188
Совмещенный (4)                       77
Раздельный (3)                        52
Совмещенный (1), Раздельный (2)       30
Совмещенный (3), Раздельный (1)       27
Совмещенный (2), Раздельный (2)       25
Раздельный (4)                        15
Совмещенный (3), Раздельный (3)        6
Совмещенный (4), Раздельный (1)        6
Совмещенный (4), Раздельный (2)        4
Совмещенный (1), Раздельный (3)        2
Совмещенный (2), Раздельный (3)        2
Совмещенный (2), Раздельный (4)        1
Совмещенный (3), Раздельный (2)        1
Name: count, dtype: int64


In [23]:
# Fill NaN values with the mode
mode_value = df['Санузел'].mode()[0]
df['Санузел'].fillna(mode_value, inplace=True)

In [24]:
print(df['Санузел'].describe)

<bound method NDFrame.describe of 0                        Совмещенный (1)
1        Совмещенный (2), Раздельный (1)
2                        Совмещенный (3)
3                        Совмещенный (3)
4                        Совмещенный (2)
                      ...               
23363                    Совмещенный (1)
23364                    Совмещенный (1)
23365                    Совмещенный (1)
23366    Совмещенный (1), Раздельный (1)
23367                     Раздельный (1)
Name: Санузел, Length: 23368, dtype: object>


In [30]:
# Create "one_merged_bathroom_only" column
df['one_merged_bathroom_only'] = df['Санузел'].apply(lambda x: 1 if x == 'Совмещенный (1)' else 0)

relative_frequency = df['one_merged_bathroom_only'].value_counts(normalize=True) * 100
print("Relative frequency of values in 'one_merged_bathroom_only':")
print(relative_frequency)

Relative frequency of values in 'one_merged_bathroom_only':
one_merged_bathroom_only
1    54.561794
0    45.438206
Name: proportion, dtype: float64


In [31]:
# Function to calculate the total number of bathrooms
def calculate_total_bathrooms(value):
    if pd.isna(value):
        return 0
    total = 0
    parts = value.split(', ')
    for part in parts:
        number = int(part.split('(')[1].strip(')')) #🚧
        total += number
    return total

# Create "many_bathrooms" column
df['many_bathrooms'] = df['Санузел'].apply(lambda x: 1 if calculate_total_bathrooms(x) > 2 else 0)

# Print the DataFrame to verify the changes
print(df[['Санузел', 'one_merged_bathroom_only', 'many_bathrooms']])

                               Санузел  one_merged_bathroom_only  \
0                      Совмещенный (1)                         1   
1      Совмещенный (2), Раздельный (1)                         0   
2                      Совмещенный (3)                         0   
3                      Совмещенный (3)                         0   
4                      Совмещенный (2)                         0   
...                                ...                       ...   
23363                  Совмещенный (1)                         1   
23364                  Совмещенный (1)                         1   
23365                  Совмещенный (1)                         1   
23366  Совмещенный (1), Раздельный (1)                         0   
23367                   Раздельный (1)                         0   

       many_bathrooms  
0                   0  
1                   1  
2                   1  
3                   1  
4                   0  
...               ...  
23363          

## Column "Лифт"
#### Create columns: "has_elevator", "has_cargo_elevator"


In [18]:
# Step 1: Create the `no_elevators` column
df['no_elevators'] = df['Лифт'].isna().astype(int)

# Step 2: Create the `has_cargo_elevator` column
df['has_cargo_elevator'] = df['Лифт'].apply(lambda x: 1 if isinstance(x, str) and 'Груз' in x else 0)

# Print the DataFrame to verify the changes
print(df[['Лифт', 'no_elevators', 'has_cargo_elevator']])

# Print the relative frequency of values
print("Relative frequency of 'no_elevators':")
print(df['no_elevators'].value_counts(normalize=True) * 100)

print("Relative frequency of 'has_cargo_elevator':")
print(df['has_cargo_elevator'].value_counts(normalize=True) * 100)

                     Лифт  no_elevators  has_cargo_elevator
0      Пасс (4), Груз (1)             0                   1
1      Пасс (1), Груз (1)             0                   1
2                Пасс (1)             0                   0
3                Пасс (1)             0                   0
4      Пасс (1), Груз (1)             0                   1
...                   ...           ...                 ...
23363  Пасс (1), Груз (1)             0                   1
23364  Пасс (1), Груз (1)             0                   1
23365                 NaN             1                   0
23366            Пасс (3)             0                   0
23367            Пасс (3)             0                   0

[23368 rows x 3 columns]
Relative frequency of 'no_elevators':
no_elevators
0    76.46354
1    23.53646
Name: proportion, dtype: float64
Relative frequency of 'has_cargo_elevator':
has_cargo_elevator
0    71.597912
1    28.402088
Name: proportion, dtype: float64


## Column "Мусоропровод"
##### fill NaN with moda



In [19]:
def show_unique_values(data, column_name):
    unique_values = data[column_name].dropna().unique()  # Remove NaN values before sorting
    sorted_values = sorted(unique_values, reverse=True)
    print(f"Unique values for '{column_name}' in descending order:", sorted_values)

# for 'Мусоропровод'
show_unique_values(df, 'Мусоропровод')

Unique values for 'Мусоропровод' in descending order: ['Нет', 'Да']


In [20]:
# 🚧 does not work for some reason, timebox for investigation is over
# #fillNa
# mode_garbage = df['Мусоропровод'].mode()
# print(mode_garbage)
# df['Мусоропровод'].fillna(mode_garbage, inplace=True)

# # df['garbage'] = df['Мусоропровод'].map({'Да': 1, 'Нет': 0})



# # # Step 4: Ensure the column type is integer
# # df['garbage'] = df['garbage'].astype(int)

# # # # Step 5: Tests
# # # relative_frequency = df['garbage'].value_counts(normalize=True) * 100
# # # print("Relative frequency of values in 'garbage':")
# # # print(relative_frequency)
# print(df['Мусоропровод'].describe())


In [32]:
print(df.dtypes)


Unnamed: 0                    int64
ID  объявления                int64
Количество комнат            object
Тип                          object
Метро                        object
Адрес                        object
Площадь, м2                  object
Дом                          object
Парковка                     object
Цена                         object
Телефоны                     object
Описание                     object
Ремонт                       object
Площадь комнат, м2           object
Балкон                       object
Окна                         object
Санузел                      object
Можно с детьми/животными     object
Дополнительно                object
Название ЖК                  object
Серия дома                   object
Высота потолков, м          float64
Лифт                         object
Мусоропровод                 object
Ссылка на объявление         object
pets_okay                   float64
kids_okay                   float64
furniture_in_the_room       